Data Preprocessing and Loading

In [3]:
from src.data.data import DataProcessor, TimeSeriesDataset, TimeSeriesDataLoader

# Preprocess data
data_processor = DataProcessor(path='data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv', 
                               start='2021-03-30 00:00:00', 
                               end='2021-03-31 00:00:00', 
                               verbose=True)
data_processor.prepare_data(exclude_columns_detrend=['SMA', 'RSI', 'MACD', 'log_ret', 'percent_ret'],
                            exclude_columns_scale=['log_ret', 'percent_ret'])

# Create dataloaders
train_loader, val_loader, test_loader = data_processor.create_dataloaders(window=10, 
                                                                          batch_size=64,
                                                                          exclude_input_columns=[''])

data_processor.train.head()